# Barlow-Beeston method for MC stats in the likelihood

Here I'm going to test out the method laid out in section 5 of Conway (arXiv:1103.0354).  The section describes a method for accounting for statistical uncertainty on MC templates in a binned MLE fit.  The idea is that there is a nuisance parameter assigned to each bin ($\beta_{i}$) that can be solved for within the likelihood using the relation,

$$
\beta^{2} + (\mu\sigma_{\beta}^{2} - 1) - n\sigma_{\beta}^{2} = 0.
$$

That is, while other n.p. and parameters of interest are determined using a numerical gradient descent, the additional n.p. corresponding to statistical uncertainty can be solved analytically (assuming a Gaussian constraint).

In [9]:
## imports and configuration
%cd '/home/naodell/work/wbr/analysis'
#%load_ext autoreload

from functools import partial
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from scipy.optimize import minimize
from tqdm import tqdm_notebook

import scripts.plot_tools as pt
import scripts.fit_helpers as fh
from nllfit.nllfitter import ScanParameters

np.set_printoptions(precision=3)
rc_params = {
             'figure.figsize': (10, 10),
             'axes.labelsize': 20,
             'axes.facecolor': 'white',
             'axes.titlesize':'x-large',
             'legend.fontsize': 20,
             'xtick.labelsize':18,
             'ytick.labelsize':18,
             'font.size':18,
             'font.sans-serif':['Arial', 'sans-serif'],
             'mathtext.sf':'Arial',
             'lines.markersize':8.,
             'lines.linewidth':2.5,
            }
matplotlib.rcParams.update(rc_params)

%connect_info

/home/naodell/work/wbr/analysis
{
  "shell_port": 36547,
  "iopub_port": 40391,
  "stdin_port": 41323,
  "control_port": 59523,
  "hb_port": 51967,
  "ip": "127.0.0.1",
  "key": "f01e5f85-0aa74a0e31bc7a90a13860ce",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-6cb0bed0-e9e7-449b-9070-b924c3fa3b10.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.


In [10]:
# configure, get the input data, and do any additional processing that is needed
input_dir  = f'local_data/templates/nominal/'
processes = ['ttbar', 't', 'ww', 'wjets', 'zjets_alt', 'diboson', 'fakes'] 
selections = [
              'ee', 'emu', 'mumu',  
              'mutau', 'etau', 
              'mu4j', 'e4j'
             ]

# initialize fit data
fit_data = fh.FitData(input_dir, selections, processes, process_cut=0.1)
params = fit_data._parameters
params_pre = fit_data.get_params_init().values.copy()

decay_map = fit_data._decay_map
ww_labels = decay_map.fancy_label.values 

In [11]:
# set up fit configuration
# bounds
bounds = [(0.1, 0.12), (0.1, 0.12), (0.1, 0.12), (0.64, 0.7)]

# minimizer options
step_sizes = 0.001*params['err_init']
step_sizes[:4] = 4*[1e-6,]
min_options = dict(maxfun=3e4, disp=None)

# Asimov dataset
sample = {cat:fit_data.mixture_model(params_pre, cat) for cat in fit_data._model_data.keys()}

# don't need to consider systematics n.p.
def reduced_objective(p, data, do_mc_stat, randomize_templates):
    new_params = np.concatenate([p, params_pre[4:]])
    fobj = partial(fit_data.objective, 
                   cost_type='poisson', 
                   no_shape=False,
                   do_mc_stat=do_mc_stat, 
                   randomize_templates=randomize_templates
                  )

    return fobj(new_params, data) 


In [12]:
# carry out the fit for baseline case w/o n.p.
fobj = partial(reduced_objective, data = sample, do_mc_stat=False, randomize_templates=False)
result_stat = minimize(fobj, params_pre[:4],
                       method = 'L-BFGS-B', 
                       options = min_options,
                       bounds = bounds,
                      )

# calculate covariance matrix from the inverse of the Hessian of the NLL
stderr_stat, _ = fh.calculate_covariance(fobj, params_pre[:4])
print(result_stat.x*100, stderr_stat*100/params_pre[:4])

[10.8 10.8 10.8 67.6] [0.109 0.093 0.518 0.078]


In [13]:
# carry out the fit with bb
fobj = partial(reduced_objective, data = sample, do_mc_stat=True, randomize_templates=False)
result_stat_mc = minimize(fobj, params_pre[:4],
                          method = 'L-BFGS-B', 
                          options = min_options,
                          bounds = bounds,
                         )

# calculate covariance matrix from the inverse of the Hessian of the NLL
stderr_mc_stat, _ = fh.calculate_covariance(fobj, params_pre[:4])
print(result_stat_mc.x*100, np.sqrt(stderr_mc_stat**2 - stderr_stat**2)*100/params_pre[:4])

[10.8 10.8 10.8 67.6] [0.076 0.065 0.439 0.068]


In [14]:
# carry out fit for n trials with random offsets for templates
ntrials = 500
results = []
cost = []
sv_accept = []
for _ in tqdm_notebook(range(ntrials)):
    
    # update random offsets
    for category, rnums in fit_data._rnum_cache.items():
        fit_data._rnum_cache[category] = 0.3*np.random.randn(rnums.size)
        
    # carry out fit
    fobj = partial(reduced_objective, data=sample, do_mc_stat=False, randomize_templates=True)
    pinit = params_pre[:4]*(1 +  0.02*np.random.randn(4)) # need this or minimizer doesn't always try
    result = minimize(fobj, pinit[:4],
                      method = 'L-BFGS-B', 
                      options = min_options,
                      bounds = bounds,
                     )
        
    print(fobj(pinit), result.fun, result.x[:4])
    if result.success:
        results.append(result.x)
        cost.append(result.fun)
    else:
        print(result)

results = np.array(results)

78516802.98133807 23.19620900912232 [0.108 0.108 0.108 0.676]
65434669.06340905 28.53068359629296 [0.108 0.108 0.108 0.676]
29486246.276996754 23.11834174551229 [0.108 0.108 0.108 0.676]
17338971.51273874 20.30416403089321 [0.108 0.108 0.108 0.676]
120897915.61061935 21.177522868016247 [0.108 0.108 0.108 0.676]
227316352.03231302 21.72760578135086 [0.108 0.108 0.108 0.676]
4716998.405900264 24.68263870029288 [0.108 0.108 0.108 0.676]
138051675.4686477 23.59849046586027 [0.108 0.108 0.108 0.676]
137562903.71047544 21.364866770592645 [0.108 0.108 0.108 0.676]
475859776.3500187 22.100940105167705 [0.108 0.108 0.108 0.676]
7823834.715770396 21.933571999270395 [0.108 0.108 0.108 0.676]
46151958.14377943 28.525255403644298 [0.108 0.108 0.108 0.676]
154231447.52649474 21.76082076176821 [0.108 0.108 0.108 0.676]
106890088.59513493 24.044519990476786 [0.108 0.108 0.108 0.676]
161660162.12113556 29.485814207880992 [0.108 0.108 0.108 0.676]
337551439.17228854 20.749425089092302 [0.108 0.108 0.108

KeyboardInterrupt: 

In [ ]:
bb_errs_toys = np.std(results - params_pre[:4], axis=0)
bb_errs_hess = np.sqrt(stderr_mc_stat**2 - stderr_stat**2)*0.3

print(bb_errs_toys, bb_errs_hess, sep='\n')

In [ ]:
# produce plot for the note
fig, axes = plt.subplots(2, 2, figsize=(12, 12), facecolor='white')
nbins = 40
labels = [r'$W\rightarrow e$', r'$W\rightarrow \mu$', r'$W\rightarrow \tau$', r'$W\rightarrow h$']

for i in range(4):
    ax = axes[i//2][i%2]
    h, _, _ = ax.hist(results[:,i], bins=nbins, alpha=0.5, label='variation trial')
    bb_mean = results[:,i].mean()
    ax.plot([bb_mean, bb_mean], [0, 1.5*h.max()], color='gray', linestyle='--', alpha=0.5, label='_nolegend_')
    ax.fill_betweenx([0, 1.5*h.max()], bb_mean - bb_errs_toys[i],  bb_mean + bb_errs_toys[i], color='grey', hatch='/', alpha=0.15, label=r'$\pm 1\sigma_{trials}$')
    
    ax.plot([params_pre[i], params_pre[i]], [0, 1.5*h.max()], color='k', linestyle='--', alpha=0.5, label='_nolegend_')
    ax.fill_betweenx([0, 1.5*h.max()], params_pre[i] - bb_errs_hess[i],  params_pre[i] + bb_errs_hess[i], color='k', hatch='-', alpha=0.15, label=r'$\pm 1\sigma_{NLL}$')
    
    ax.text(0.05, 0.9, f'trials: {100*bb_mean:.2f} +- {100*bb_errs_toys[i]:.3f}', fontsize=12, transform=ax.transAxes)
    ax.text(0.05, 0.85, f'BB: {100*params_pre[i]:.2f} +- {100*bb_errs_hess[i]:.3f}', fontsize=12, transform=ax.transAxes)
    
    ax.set_xbound(params_pre[i] - 5*bb_errs_hess[i], params_pre[i] + 5*bb_errs_hess[i])
    ax.set_ybound(0., 1.4*h.max())
    ax.set_xlabel(labels[i])
    
    if i%2 == 0:
        ax.set_ylabel('Entries')
        
    if i == 3:
       plt.legend() 

plt.tight_layout()
plt.show()